<a href="https://colab.research.google.com/github/Myrto-Iglezou/AI2-project4/blob/master/Question_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## YΣ19 Artificial Intelligence II
# Homework 4

### Iglezou Myrto - 111520170038

# Project Description



Build a BERT-based model which returns “an answer”, given a user question and a
passage which includes the answer of the question. For this question answering task, we
will use the SQuAD 2.0 dataset. We will start with the BERT-base pretrained model “bert-base-uncased”
and fine-tune it to have a question answering task.


# **Question 3** 

### Load SCuAD 2.0 dataset from drive

In [17]:
import io
import os
from google.colab import drive
import pandas as pd 
import json
import sys

drive.mount('/content/drive',force_remount=True)

sys.path.append('/content/drive/My Drive/')

!cp -r "/content/drive/My Drive/train-v2.0.json" '/content/'
!cp -r "/content/drive/My Drive/dev-v2.0.json" '/content/'
!cp -r "/content/drive/My Drive/utils_squad.py" '/content/'
!cp -r "/content/drive/My Drive/utils_squad_evaluate.py" '/content/'

squad_file = '/content/train-v2.0.json'
validation_file = '/content/dev-v2.0.json'

Mounted at /content/drive


In [18]:
%%capture
!pip install pytorch-pretrained-bert pytorch-nlp pytorch_transformers

In [19]:
%%capture
!pip install transformers

In [20]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertForQuestionAnswering

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


**utils_squad** is a library that helps handle the SQuAD dataset

In [21]:
from utils_squad import (read_squad_examples, convert_examples_to_features,
                         RawResult, write_predictions,
                         RawResultExtended, write_predictions_extended)
from utils_squad_evaluate import EVAL_OPTS, main as evaluate_on_squad, plot_pr_curve

In [22]:
data = read_squad_examples(input_file=squad_file,
                                is_training=True,
                                version_2_with_negative=True)

val_data = read_squad_examples(input_file=validation_file,
                                is_training=False,
                                version_2_with_negative=True)

In [23]:
train_data = pd.DataFrame.from_records([vars(line) for line in data])
train_data.head()

,qas_id,question_text,doc_tokens,orig_answer_text,start_position,end_position,is_impossible
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...",in the late 1990s,39,42,False
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...",singing and dancing,28,30,False
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...",2003,82,82,False
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...","Houston, Texas",22,23,False
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,"[Beyoncé, Giselle, Knowles-Carter, (/biːˈjɒnse...",late 1990s,41,42,False


In [24]:
doc_stride = 64
max_seq_length = 128
max_query_length = 32
batch_size = 16

In [ ]:
features = convert_examples_to_features(examples=data,
                                        tokenizer=tokenizer,
                                        max_seq_length=max_seq_length,
                                        doc_stride=doc_stride,
                                        max_query_length=max_query_length,
                                        is_training=True)

val_features = convert_examples_to_features(examples=val_data,
                                        tokenizer=tokenizer,
                                        max_seq_length=max_seq_length,
                                        doc_stride=doc_stride,
                                        max_query_length=max_query_length,
                                        is_training=False)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)

all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)
dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                        all_start_positions, all_end_positions,
                        all_cls_index, all_p_mask)

In [ ]:
# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in val_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in val_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in val_features], dtype=torch.long)
all_cls_index = torch.tensor([f.cls_index for f in val_features], dtype=torch.long)
all_p_mask = torch.tensor([f.p_mask for f in val_features], dtype=torch.float)

all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
val_dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                        all_example_index, all_cls_index, all_p_mask)

In [ ]:
train_sampler = RandomSampler(dataset)
train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size, drop_last=True)

validation_sampler = SequentialSampler(dataset)
validation_dataloader = DataLoader(val_dataset, sampler=validation_sampler, batch_size=batch_size, drop_last=True)

In [ ]:
import torch
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
    device = 'cuda'
else:
    print('No GPU available, training on CPU.')
    device = 'cpu'

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased').to(device=device)
param_optimizer = list(model.named_parameters())

In [ ]:
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = torch.optim.Adam(lr=1e-5, betas=(0.9, 0.98), eps=1e-9, params=optimizer_grouped_parameters)

In [ ]:
epochs = 4

for epoch in range(1, epochs + 1):
    # ============================================ TRAINING ============================================================
    print("Training epoch ", str(epoch))
   
    model.train()
    tr_loss = 0
    nb_tr_steps = 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)

        inputs = {'input_ids':       batch[0],
                  'attention_mask':  batch[1], 
                  'token_type_ids':  batch[2],  
                  'start_positions': batch[3], 
                  'end_positions':   batch[4]}

        optimizer.zero_grad()

        outputs = model(**inputs)
        loss = outputs[0]

        loss.backward()
        optimizer.step()
        tr_loss += loss.item()
        nb_tr_steps += 1

    print(f"\nTraining loss={tr_loss / nb_tr_steps:.4f}")

    # ============================================ VALIDATION ==========================================================
    model.eval()
    currentQuestion = 0
    correct_answers = 0
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        
        with torch.no_grad():
          inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'token_type_ids': batch[2]
                    }
          example_indices = batch[3]
          outputs = model(**inputs)
            # pred_start, pred_end = outputs[0].detach().cpu().numpy(), outputs[1].detach().cpu().numpy()

        for i, example_index in enumerate(example_indices):
          eval_feature = features[example_index.item()]
          unique_id = int(eval_feature.unique_id)

          result = RawResult(unique_id    = unique_id,
                            start_logits = to_list(outputs[0][i]),
                            end_logits   = to_list(outputs[1][i]))
          all_results.append(result)


      output_prediction_file = "/drive/My Drive/predictions.json"
      output_nbest_file = "/drive/My Drive/nbest_predictions.json"
      output_null_log_odds_file = "/drive/My Drive/null_odds.json"
      output_dir = "/drive/My Drive/predict_results"

      write_predictions(val_examples, features, all_results, 10,
                      30, True, output_prediction_file,
                      output_nbest_file, output_null_log_odds_file, False,
                      True, 0.0)

      evaluate_options = EVAL_OPTS(data_file=predict_file,
                                  pred_file=output_prediction_file,
                                  na_prob_file=output_null_log_odds_file,
                                  out_image_dir=None)
      results = evaluate_on_squad(evaluate_options)



    #     for question, (start, end) in enumerate(zip(pred_start, pred_end)):
    #         eval_feature = val_features[example_index.item()]
    #         currentQuestion += 1
    #         offsets = squad_eg.context_token_to_char
    #         start = np.argmax(start)
    #         end = np.argmax(end)
    #         if start >= len(offsets):
    #             continue
    #         pred_char_start = offsets[start][0]
    #         if end < len(offsets):
    #             pred_char_end = offsets[end][1]
    #             pred_ans = squad_eg.context[pred_char_start:pred_char_end]
    #         else:
    #             pred_ans = squad_eg.context[pred_char_start:]
    #         normalized_pred_ans = normalize_text(pred_ans)
    #         normalized_true_ans = [normalize_text(_) for _ in squad_eg.all_answers]
    #         if normalized_pred_ans in normalized_true_ans:
    #             correct_answers += 1
    # acc = correct_answers / len(val_data)
    # print(f"\nAccuracy score={acc:.2f}")

In [ ]:
OPTS = evaluate_options
with open(OPTS.data_file) as f:
  dataset_json = json.load(f)
  dataset = dataset_json['data']
with open(OPTS.pred_file) as f:
  preds = json.load(f)
if OPTS.na_prob_file:
  with open(OPTS.na_prob_file) as f:
    na_probs = json.load(f)

In [ ]:
exact_raw, f1_raw = get_raw_scores(dataset, preds)

## References



*   https://github.com/nlpyang/pytorch-transformers/tree/master/examples

